# Evaluation of the final CNN model
The model was chosen in 7_CNN_model_comparison.ipynb and the hyperparameters chosen from 8_CNN_hyperparamete.ipynb.

First the model is trained on subjects A, B, D, E, F.
The model is then evaluated on the test subject C. Up until this point the model has not been exposed to this data.

## Notebook setup

Allow editing of modules using editor (auto reloading)

In [1]:
# Needed to allow editing using PyCharm etc
%load_ext autoreload
%autoreload 2

The following cell is needed for compatibility when using both CoLab and Local Jupyter notebook. It sets the appropriate file path for the data and also installs local packages such as models and data_loading.

In [2]:
import os
path = os.getcwd()
if path == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    BASE_PATH = '/content/gdrive/My Drive/Level-4-Project/'
#     !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
elif path == 'D:\\Google Drive\\Level-4-Project\\notebooks':
    BASE_PATH = "D:/Google Drive/Level-4-Project/"
    
elif path == "/export/home/2192793m":
    BASE_PATH = "/export/home/2192793m/Level-4-Project/"
    
DATA_PATH = BASE_PATH + 'data/processed/doppler_spectrograms/3/'

RESULTS_PATH = BASE_PATH + 'results/CNN_final_model_evaluation/'
HYPERPARAMETER_PATH = BASE_PATH + 'results/CNN_hyperparameter_search/'
MODEL_PATH = BASE_PATH + 'models/CNN_final_model_evaluation/'

if not os.path.exists(RESULTS_PATH):
    os.makedirs(RESULTS_PATH)
    
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

Import remaining packages

In [3]:
import numpy as np
import sys
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
import sys
from sklearn.metrics import classification_report, confusion_matrix
import csv
from keras.models import load_model
import pickle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D

Using TensorFlow backend.


In [4]:
# ! pip install scikit-optimize
from skopt import load
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args

In [5]:
# Needed as originally code was for theano backend but now using tensor flow
from keras import backend as K
K.set_image_dim_ordering('th')

## Experiment Setup

In [6]:
target_names = ["walking", "pushing", "sitting", "pulling", "circling", "clapping", "bending"]
nb_classes = len(target_names)

# input image dimensions
img_rows, img_cols = 75, 75

users = ["A", "B", "C", "D", "E", "F"] 

In [7]:
def load_data(user_letter):
    with open(DATA_PATH + user_letter + "_data.pkl", 'rb') as data_file:
        data = pickle.load(data_file)
        data = data.reshape(data.shape[0], 1, 75, 75)
        
    with open(DATA_PATH + user_letter + "_labels.pkl", 'rb') as labels_file:
        labels = pickle.load(labels_file)
        labels = np.reshape(labels, (len(labels), 1))

        
    return data, labels

In [9]:
datasets = {}
for user in users:
    data, labels = load_data(user)
    datasets[user] = {"data":data, "labels":labels}

In [8]:
def split_train_validation(validation_user):
    train_data = None
    train_labels = None
    first_round = True
    validation_data = []
    validation_labels = []
    for user in users:
        data = datasets[user]["data"]
        labels = datasets[user]["labels"]
        if user == validation_user:
            validation_data = data
            validation_labels = labels
            
        else:
            if first_round:
                train_data = data
                train_labels = labels
                first_round = False
            else:
                train_data = np.concatenate((train_data, data))
                train_labels = np.concatenate((train_labels, labels))
            
    train_labels = np_utils.to_categorical(train_labels, nb_classes)
    validation_labels = np_utils.to_categorical(validation_labels, nb_classes)
    train_data = train_data.astype('float32')
    validation_data = validation_data.astype('float32')
    train_data /= 255
    validation_data /= 255 
    
    return {
        "train_data": train_data,
        "train_labels": train_labels,
        "validation_data": validation_data,
        "validation_labels": validation_labels
       }

## Define Model

In [10]:
def make_model(nb_filters, img_rows, img_cols, nb_classes, activation,
               dropout, num_dense_nodes, num_dense_layers,
               kernel_size, pooling_size):
    
    kernel_size = (kernel_size, kernel_size)
    pooling_size = (pooling_size, pooling_size)
    
    model = Sequential(name=nb_filters)
    nb_filters = nb_filters.split("-")
    size_1 = int(nb_filters[0])
    size_2 = int(nb_filters[1])

    model.add(Convolution2D(size_1, kernel_size, padding='same', input_shape=(1, img_rows, img_cols), activation=activation))
    model.add(Convolution2D(size_1, kernel_size, activation=activation))
    model.add(MaxPooling2D(pool_size=pooling_size))
    model.add(Dropout(dropout))

    model.add(Convolution2D(size_2, kernel_size, padding='same', activation=activation))
    model.add(Convolution2D(size_2, kernel_size, activation=activation))
    model.add(MaxPooling2D(pool_size=pooling_size))
    model.add(Dropout(dropout))

    model.add(Flatten())
    for i in range(num_dense_layers):
        model.add(Dense(num_dense_nodes, activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(nb_classes, activation='softmax'))
    return model

### Load hyperparameter results

In [12]:
# Need objective function to load data
space = [
    Categorical(['adam', 'sgd_standard', 'sgd_nestrov'], name='optimizer'),
    Real(0.0001, 0.1, "log-uniform", name='learning_rate'),
    Categorical(['relu', 'sigmoid', 'tanh'], name='activation'),
    Real(0.1, 0.9, name='dropout'),
    Integer(16, 1024, name='num_dense_nodes'),
    Integer(1,3, name='num_dense_layers'),
    Integer(2,5, name='kernel_size'),
    Integer(2,4, name='pooling_size'),
    Integer(8, 1024, name='batch_size')
]

@use_named_args(space)
def objective(**params):
    average_accuracy = 0
    average_loss = 0
    for user in users:
        data_split = split_train_validation(user)
        train_data = data_split["train_data"]
        train_labels = data_split["train_labels"]
        validation_data = data_split["validation_data"]
        validation_labels = data_split["validation_labels"]

        model = make_model("8-16", img_rows, img_cols, nb_classes,
                           params["activation"], params['dropout'],
                           params['num_dense_nodes'],
                           params['num_dense_layers'], params['kernel_size'],
                           params['pooling_size'])
        if params['optimizer'] == 'adam':
            selected_optimizer = Adam(lr=params['learning_rate'])
            
        elif params['optimizer'] == 'sgd_standard':
            selected_optimizer = SGD(lr=params['learning_rate'])
            
        else:
            #nestrov momentum
            selected_optimizer = SGD(lr=params['learning_rate'], decay=1e-6, 
                                     momentum=0.9, nesterov=True)
            
        model.compile(loss='categorical_crossentropy',
                      optimizer=selected_optimizer,
                      metrics=['accuracy'])

        model.fit(
            train_data,
            train_labels,
            batch_size=params['batch_size'],
            epochs=nb_epoch,
            shuffle=True, 
            verbose=1)

        evaluation = model.evaluate(validation_data, validation_labels,
                                    batch_size=params['batch_size'], verbose=0)

        average_loss += evaluation[0]
        average_accuracy += evaluation[1]
        
    return -(average_accuracy/len(users)) # return negative as minimizing

In [13]:
dimensions = ['optimizer', 'learning_rate', 'activation', 'dropout',
              'num_dense_nodes', 'num_dense_layers', 'kernel_size',
              'pooling_size','batch_size']
# res_gp = load(HYPERPARAMETER_PATH + "res_gp_complete.pkl")
res_gp = load(HYPERPARAMETER_PATH + "res_gp_checkpoint.pkl") # used for testing
parameters = res_gp.x

C:\Users\macka\Anaconda3\envs\dataScience\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.20.2 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [14]:
for index, parameter in enumerate(parameters):
    print(dimensions[index] + ":", parameter)

optimizer: adam
learning_rate: 0.0022567198219888875
activation: relu
dropout: 0.4533687369990764
num_dense_nodes: 1003
num_dense_layers: 2
kernel_size: 3
pooling_size: 3
batch_size: 903


## Train Model

In [15]:
nb_epoch = 20

In [16]:
model = make_model("8-16", img_rows, img_cols, nb_classes, parameters[2],
               parameters[3], parameters[4], parameters[5],
               parameters[6], parameters[7])

In [17]:
data_split = split_train_validation("C") # subject c is test subject
train_data = data_split["train_data"]
train_labels = data_split["train_labels"]
test_data = data_split["validation_data"]
test_labels = data_split["validation_labels"]

In [18]:
optimizer = parameters[0] 
learning_rate = parameters[1]
if optimizer == 'adam':
    selected_optimizer = Adam(lr=learning_rate)

elif optimizer == 'sgd_standard':
    selected_optimizer = SGD(lr=learning_rate)

else:
    #nestrov momentum
    selected_optimizer = SGD(lr=learning_rate, decay=1e-6, momentum=0.9,
                             nesterov=True)
               
model.compile(loss='categorical_crossentropy',
              optimizer=selected_optimizer,
              metrics=['accuracy'])

In [20]:
history = model.fit(train_data, train_labels,
                    batch_size=parameters[8],
                    epochs=nb_epoch,
                    shuffle=True, 
                    validation_data=(test_data, test_labels),
                    verbose=1)

Train on 36985 samples, validate on 7966 samples
Epoch 1/20
36985/36985 [==============================] - 52s 1ms/step - loss: 1.2550 - acc: 0.5119 - val_loss: 0.9298 - val_acc: 0.5743
Epoch 2/20
36985/36985 [==============================] - 31s 849us/step - loss: 0.5711 - acc: 0.7985 - val_loss: 0.9800 - val_acc: 0.5796
Epoch 3/20
36985/36985 [==============================] - 32s 860us/step - loss: 0.3526 - acc: 0.8770 - val_loss: 0.9160 - val_acc: 0.6693
Epoch 4/20
36985/36985 [==============================] - 32s 853us/step - loss: 0.2585 - acc: 0.9068 - val_loss: 0.9200 - val_acc: 0.6666
Epoch 5/20
36985/36985 [==============================] - 32s 859us/step - loss: 0.2248 - acc: 0.9192 - val_loss: 0.9239 - val_acc: 0.6864
Epoch 6/20
36985/36985 [==============================] - 32s 866us/step - loss: 0.1959 - acc: 0.9289 - val_loss: 1.0098 - val_acc: 0.6622
Epoch 7/20
36985/36985 [==============================] - 32s 863us/step - loss: 0.1755 - acc: 0.9348 - val_loss: 0.916

## Evaluate Model

In [21]:
evaluation = model.evaluate(test_data, test_labels,
                            batch_size=parameters[8], verbose=1)

loss = evaluation[0]
accuracy = evaluation[1]

test_pred = model.predict_classes(test_data)
report = classification_report(np.argmax(test_labels,axis=1),
                               test_pred, target_names=target_names)
conf_matrix = confusion_matrix(np.argmax(test_labels,axis=1), test_pred)

7966/7966 [==============================] - 2s 288us/step


## Save Model

In [0]:
model.save(MODEL_PATH + "final_trained_model.h5")

## Save Results

In [0]:
results = {
    "history": history.history,
    "loss": loss,
    "accuracy": accuracy,
    "confusion_matrix": conf_matrix,
    "classification_report": report
}
with open(RESULTS_PATH + "results_dictionary.pkl", 'wb') as results_file:
    pickle.dump(results, results_file)

## Load Results

In [0]:
with open(RESULTS_PATH + "results_dictionary.pkl", 'rb') as results_file:
    results = pickle.load(results_file)

## Visualize Results

### Accuracy

In [22]:
training_acc = results["history"]['acc']
test_acc = results["history"]['val_acc']

# Create count of the number of epochs
epoch_count = range(1, len(training_acc) + 1)

# Visualize loss history
plt.plot(epoch_count, training_acc, 'r--')
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Training Accuracy', 'Test Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title("Comparison of Training vs Test Accuracy")
if True:
    plt.savefig(RESULTS_PATH + "training_vs_test_acc.pdf", format='pdf')
plt.show()

NameError: name 'results' is not defined

### Loss

In [0]:
training_loss = results["history"]['loss']
test_loss = results["history"]['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("Comparison of Training vs Test Loss")
if True:
    plt.savefig(RESULTS_PATH + "training_vs_test_loss.pdf", format='pdf')
plt.show()

### Confusion Matrix

In [0]:
def plot_confusion_matrix(cm, classes, save=False, path='/'):
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    if save:
        plt.savefig(path, format='pdf')
    plt.show()

In [0]:
plot_confusion_matrix(conf_matrix, target_names,
                      save=True, path=RESULTS_PATH + "confusion_matrix.pdf")